# network_score
Takes a network and returns a list of its scores

## Requirements:
* The network being scored must be open in Cytoscape Desktop
* The file `cyto_to_shapely_node_util.ipynb` must be run
* The file `cyto_to_shapely_edge_util.ipynb` must be run
* The file `full_cyto_to_shapely_util.ipynb` must be run

## Methods:
### network_score(network_suid, view_suid, x_lim_range_max, y_lim_range_max, edge_length_threshold=35, edge_direction='down', edge_angle_threshold=15)
#### Parameters:
* network_suid: Suid of network to score
* view_suid: View suid of network to score
* x_lim_range_max: Maximum x direction bounds for network 
* y_lim_range_max: Maximum y direction bounds for network
* edge_length_threshold: Minimum edge length that is not penalized for being too short. Default: 35
* edge_direction: Expected direction of edges. Either 'up' or 'down'. Default: 'down'
* edge_angle_threshold: Number of degrees above/below the horizontal that an edge has to be to be considered going up/down. Default: 15

#### Returns:
* List of scores in the following order:
    * Edge crossings
    * Node overlaps
    * Edge length
    * Node distance
    * Node-edge distance
    * Downward edge
    
#### Effects:
* None

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np

def network_score(network_suid, view_suid, x_lim_range_max, y_lim_range_max, edge_length_threshold=35, edge_direction='down', edge_angle_threshold=15):
    scores = []
    
    #Import network as shapely shapes and lines
    shapes, lines = full_cyto_to_shapely(network_suid, view_suid)

    #View network and get limits
    plt.clf()
    fig = plt.figure(1, figsize=(5,5), dpi=90)
    ax = fig.add_subplot(111)
    coords = []
    for shape_info in shapes.values():
        coords.append(shape_info['shape'].exterior.xy)
    for coord in coords:
        x, y = coord
        ax.plot(x, y, linewidth=0.5)
    for line_info in lines.values():
        line = line_info['line']
        ax.plot(*np.array(line).T, linewidth=0.5)
    ax.set_aspect(1)
    ax.set_ylim(ax.get_ylim()[::-1])
    
    #Check boundaries
    xlim = ax.get_xlim()    
    ylim = ax.get_ylim()

    if abs(xlim[0] - xlim[1]) > x_lim_range_max:
        print('x')
        print(xlim)
        return [0] * 7
    if abs(ylim[0] - ylim[1]) > y_lim_range_max:
        print('y')
        print(ylim)
        return [0] * 7
    diagonal = sqrt(x_lim_range_max**2 + y_lim_range_max**2)
    
    #Get scores
    scores.append(_get_edge_crossing_score(lines))
    scores.append(_get_node_overlap_score(shapes))
    scores.append(_get_node_edge_overlap_score(shapes, lines))
    scores.append(_get_edge_length_score(lines, diagonal, edge_length_threshold))
    scores.append(_get_node_distance_score(shapes, diagonal))
    scores.append(_get_node_edge_distance_score(shapes, lines, diagonal))
    scores.append(_get_downward_edge_score(shapes, lines, edge_direction, edge_angle_threshold))
    
    return scores
    
def _get_edge_crossing_score(lines):
    edge_crossings = 0

    checked = []
    for outer_line_suid, outer_line_info in lines.items():
        checked.append(outer_line_suid)
        outer_line = outer_line_info['line']
        for inner_line_suid, inner_line_info in lines.items():
            if inner_line_suid in checked:
                continue
            inner_line = inner_line_info['line']
            if inner_line.intersects(outer_line):
                edge_crossings += 1

    possible_edge_crossings = len(checked) * (len(checked) - 1) / 2
    edge_crossings_score = (possible_edge_crossings - edge_crossings) / possible_edge_crossings
    return edge_crossings_score

def _get_node_overlap_score(shapes):
    node_overlaps = 0

    checked = []
    for outer_shape_suid, outer_shape_info in shapes.items():
        checked.append(outer_shape_suid)
        outer_shape = outer_shape_info['shape']
        for inner_shape_suid, inner_shape_info in shapes.items():
            if inner_shape_suid in checked:
                continue
            inner_shape = inner_shape_info['shape']
            if inner_shape.intersects(outer_shape):
                node_overlaps += 1

    possible_node_overlaps = len(checked) * (len(checked) - 1) / 2
    node_overlaps_score = (possible_node_overlaps - node_overlaps) / possible_node_overlaps
    return node_overlaps_score

def _get_node_edge_overlap_score(shapes, lines):
    node_edge_overlaps = 0

    for shape_suid, shape_info in shapes.items():
        shape = shape_info['shape']
        for line_info in lines.values():
            if shape_suid == line_info['source'] or shape_suid == line_info['target']:
                continue
            if line_info['line'].intersects(shape):
                node_edge_overlaps += 1

    possible_node_edge_overlaps = (len(shapes) - 2) * len(lines)
    node_edge_overlaps_score = (possible_node_edge_overlaps - node_edge_overlaps) / possible_node_edge_overlaps
    return node_edge_overlaps_score

def _get_edge_length_score(lines, diagonal, edge_length_threshold):
    total_edge_cost = 0

    for line_info in lines.values():
        total_edge_cost += _cost(line_info['line'].length, diagonal, edge_length_threshold)

    edge_cost_score = max(0, 1 - total_edge_cost/len(lines)/diagonal)
    return edge_cost_score

def _cost(length, diagonal, edge_length_threshold):#Can have penalty for extremely short edges
    if length < edge_length_threshold:
        return diagonal
    return length


def _get_node_distance_score(shapes, diagonal):
    total_node_distance = 0

    for outer_shape_suid, outer_shape_info in shapes.items():
        outer_shape = outer_shape_info['shape']
        min_distance = float('Infinity')
        for inner_shape_suid, inner_shape_info in shapes.items():
            inner_shape = inner_shape_info['shape']
            if inner_shape == outer_shape:
                continue
            if inner_shape_suid in outer_shape_info['neighbors']:
                continue
            distance = outer_shape.distance(inner_shape)
            if distance < min_distance:
                min_distance = distance
        total_node_distance += min_distance

    node_distance_score = total_node_distance / diagonal / len(shapes)
    return node_distance_score

def _get_node_edge_distance_score(shapes, lines, diagonal):
    total_node_edge_distance = 0

    for shape_suid, shape_info in shapes.items():
        shape = shape_info['shape']
        min_distance = float('Infinity')
        for line_info in lines.values():
            if line_info['source'] == shape_suid or line_info['target'] == shape_suid:
                continue
            distance = shape.distance(line_info['line'])
            if distance < min_distance:
                min_distance = distance
        total_node_edge_distance += min_distance
    
    node_edge_distance_score = total_node_edge_distance / diagonal / len(shapes)
    return node_edge_distance_score

def _get_downward_edge_score(shapes, lines, edge_direction, edge_angle_threshold):
    downward_edges = 0
    
    for line_info in lines.values():
        source_suid = line_info['source']
        target_suid = line_info['target']
        if _is_downward_pointing(
            shapes[source_suid]['x'],
            shapes[source_suid]['y'],
            shapes[target_suid]['x'],
            shapes[target_suid]['y'],
            edge_direction,
            edge_angle_threshold
        ):
            downward_edges += 1

    downward_edges_score = downward_edges / len(lines)
    return downward_edges_score

def _is_downward_pointing(source_x, source_y, target_x, target_y, edge_direction, edge_angle_threshold):
    if edge_direction == 'down':
        if source_y > target_y:
            return False
    else:
        if source_y < target_y:
            return False
    dx = abs(target_x - source_x)
    dy = abs(target_y - source_y)
    if dx == 0:
        return True
    if math.atan(dy / dx) > (edge_angle_threshold * math.pi / 180):
        return True
    return False